# NextGenFace: 3D Face Reconstruction with Mitsuba 3

This notebook demonstrates 3D face reconstruction using differentiable rendering with Mitsuba 3.

## Setup

1. Enable GPU: **Runtime > Change runtime type > GPU**
2. Run the cell below to clone and install
3. **Restart the runtime** when prompted (Runtime > Restart session)
4. Continue from the "Upload Model Data" cell

In [ ]:
!git clone https://github.com/suvojit-0x55aa/NextGenFace.git
%cd NextGenFace
!pip install -e .

In [ ]:
import os
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Image
from google.colab import files

## Upload Model Data

You must upload these files (they'll be placed in `data/baselMorphableModel/`):

1. **Basel Face Model**: `model2017-1_face12_nomouth.h5` from https://faces.dmi.unibas.ch/bfm/bfm2017.html
2. **Albedo Model**: `albedoModel2020_face12_albedoPart.h5` from https://github.com/waps101/AlbedoMM/releases

These files require agreeing to license terms and cannot be redistributed.

In [ ]:
# After runtime restart, working directory resets to /content
%cd /content/NextGenFace

# Set Mitsuba variant for Colab (GPU rendering with CUDA)
os.environ["NEXTGENFACE_MITSUBA_VARIANT"] = "cuda_ad_rgb"

# Upload model files to data/baselMorphableModel/
upload_dir = 'data/baselMorphableModel'
os.makedirs(upload_dir, exist_ok=True)

print("Upload model files:")
print("  1. model2017-1_face12_nomouth.h5")
print("  2. albedoModel2020_face12_albedoPart.h5")
print()

uploaded = files.upload()
for filename in uploaded:
    dest = os.path.join(upload_dir, filename)
    shutil.move(filename, dest)
    print(f"Saved: {dest}")

## Run Reconstruction

In [ ]:
from optim.optimizer import Optimizer
from optim.config import Config

config = Config()
config.fillFromDicFile('configs/default.ini')
config.device = 'cuda'

# GPU can handle more iterations for better quality
config.iterStep1 = 5000
config.iterStep2 = 1000
config.iterStep3 = 1500
config.rtSamples = 20000

config.print()

In [ ]:
imagePath = 'data/input/s1.png'  # path to input image
outputDir = 'data/output/' + os.path.basename(imagePath.strip('/'))

optimizer = Optimizer(outputDir, config)
optimizer.run(imagePath)

## View Results

In [ ]:
def show_results(output_dir):
    """Display render output and loss curves for a reconstruction."""
    display(Image(os.path.join(output_dir, 'render_0.png')))

    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    stage_titles = [
        'Stage 1: Landmarks (Pose + Expression)',
        'Stage 2: Shape + Albedo + Light',
        'Stage 3: Texture Refinement'
    ]
    for idx, (ax, title) in enumerate(zip(axes, stage_titles)):
        loss_path = os.path.join(output_dir, 'checkpoints', f'stage{idx+1}_loss.png')
        if os.path.exists(loss_path):
            img = mpimg.imread(loss_path)
            ax.imshow(img)
        else:
            ax.text(0.5, 0.5, 'Not available', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

show_results(outputDir)

## Try Your Own Image

Upload a face image and run the reconstruction pipeline on it.

In [ ]:
print("Upload a face image (jpg/png):")
uploaded = files.upload()

custom_image_path = None
os.makedirs('data/input', exist_ok=True)
for filename in uploaded:
    dest = os.path.join('data/input', filename)
    shutil.move(filename, dest)
    custom_image_path = dest
    print(f"Saved: {dest}")

In [ ]:
customOutputDir = 'data/output/' + os.path.splitext(os.path.basename(custom_image_path))[0]

custom_optimizer = Optimizer(customOutputDir, config)
custom_optimizer.run(custom_image_path)

In [ ]:
show_results(customOutputDir)